In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from fractions import Fraction
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pennylane as qml
from brokenaxes import brokenaxes
from matplotlib.colors import LogNorm
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from matplotlib.ticker import FuncFormatter, LogLocator, MultipleLocator
from mpl_toolkits.axes_grid1 import make_axes_locatable
from my_favorite_things import (
    bar_count,
    format_ddict,
    multifader,
    nested_ddict,
    pprint_nested_dict,
)
from pennylane import numpy as qmlnp
from scipy.optimize import curve_fit

from scripts.constants import INVMS, NUM_FSP_DICT, SYM_TRUE_BS_DICT, TOP_MASS, W_MASS
from scripts.data import get_bitstring_invms, split_data
from scripts.events import get_data
from scripts.files import load_data, parse_data, verify_data
from scripts.hamiltonians import (
    get_all_bitstring_energies,
    get_bitstring_energies,
    get_coefficients,
    get_minimum_energies,
    swap,
)
from scripts.hemisphere import run_hemisphere
from scripts.pennylane_algs import MAQAOA, VarQITE
from scripts.postdata import (
    create_falqon_depth,
    find_efficiency,
    get_2dhist_invms,
    get_bitstrings,
    parse_with_metadata,
)

In [ ]:
# There's a deprecation warning for functools.partial from pennylane and it's just
# annoying to see everytime, so we silence it for now
import warnings

warnings.filterwarnings(
    "ignore",
    message=".*functools.partial will be a method descriptor.*",
    category=FutureWarning,
)

# Print out numpy arrays to be more readable
np.set_printoptions(linewidth=200)

# Matplotlib formatting params
plt.rcParams.update(
    {
        "axes.labelsize": 24,
        "ytick.labelsize": 18,
        "xtick.labelsize": 18,
        "figure.subplot.wspace": 0.25,
        "figure.subplot.hspace": 0.2,
        "axes.titlesize": 26,
        "legend.fontsize": 15,
    }
)

# Fill for Hamiltonian limit
FILL_KWARGS = {"color": "#B2B2BF", "hatch": "x", "alpha": 0.3}
COLORS = {
    "qaoa": "#68C3D4",
    "maqaoa": "#E84B45",
    "xqaoa": "#762FCD",
    "falqon": "#E3B505",
    "varqite": "#2F923C",
    "hemisphere": "#101010",
    "annealer": "#A7A78B",
}
LSS = {
    "qaoa": "solid",
    "maqaoa": "solid",
    "xqaoa": "solid",
    "falqon": "solid",
    "varqite": "solid",
    "hemisphere": "dashdot",
    "annealer": "dashed",
}
LABELS = {
    "qaoa": "QAOA",
    "maqaoa": "ma-QAOA",
    "xqaoa": "XQAOA",
    "falqon": "FALQON",
    "varqite": "VarQITE",
    "hemisphere": "Hemisphere Method",
    "annealer": "Quantum Annealer",
}

---

# Loading Data

In [ ]:
evts_etype = "ttbar"
evts_dtype = "parton"
CORRECT = SYM_TRUE_BS_DICT[evts_etype]

evts, Jijs, Pijs, invms = get_data(etype=evts_etype, dtype=evts_dtype)
split_evts, split_inds = split_data(evts=evts)
split_Jijs = Jijs[split_inds]
split_Pijs = Pijs[split_inds]
split_invms = invms[split_inds]
split_H0_coeffs = get_coefficients(hamiltonian="H0", evts=evts)[split_inds]
split_H2_coeffs = get_coefficients(
    hamiltonian="H2", evts=evts, nume=["min", "Jij"], denom=["max", "Pij"]
)[split_inds]

bruteforce_dict = {
    "H0": [
        (get_minimum_energies(evts, hamiltonian="H0")[:, 0] == CORRECT).sum()
        / split_evts.shape[1]
        for evts in split_evts
    ],
    "H2": [
        (
            get_minimum_energies(
                evts, hamiltonian="H2", nume=["min", "Jij"], denom=["max", "Pij"]
            )[:, 0]
            == CORRECT
        ).sum()
        / split_evts.shape[1]
        for evts in split_evts
    ],
}

In [ ]:
# Note: the FALQON data takes up ~100x more space because data is saved
# for every of the 2500 depths, so we choose the depths we want
falqon_depths = np.sort(
    np.concatenate(
        (
            np.arange(0, 1001, 100)[1:],
            np.arange(1000, 2501, 500),
            [25, 50, 150, 250, 350, 450, 750],
        )
    )
)
num_runs = len(parse_data())
data = []
metadata = []
for ind, metadatum in enumerate(parse_data()):
    print(" " * 50, end="\r")
    print(
        f"Loading data [{metadatum[0]}]: {ind + 1:>{len(str(num_runs))}} / {num_runs}",
        end="\r",
    )
    if not (verify_data(*metadatum)):
        print(f"{metadatum}\nSkipping!\n")
        continue

    datum = load_data(*metadatum)
    if metadatum[0] == "falqon":
        for depth in falqon_depths:
            new_datum, new_metadatum = create_falqon_depth(
                datum=datum, metadatum=metadatum, depth=depth
            )
            data.append(new_datum)
            metadata.append(new_metadatum)
    else:
        data.append(datum)
        metadata.append(metadatum)

data = np.array(data)
metadata = np.array(metadata)

In [ ]:
def sizeof(datum):
    tot_space = 0
    for invmd in datum.values():
        for val in invmd.values():
            tot_space += val.nbytes
    return tot_space


tot_total = 0
for datum, metadatum in zip(data, metadata):
    size = sizeof(datum)
    tot_total += size
    print(f"{metadatum[0]:<7} ({metadatum[1]:>4}) -- {size / 1e6:>10,.2f} MB")
print("-" * 30 + f"\nTotal -- {tot_total / 1e9:,.2f} GB")

In [ ]:
d = nested_ddict(3, list)
for metadatum in metadata:
    alg = metadatum[0]
    ham = metadatum[2]
    evt = f"{metadatum[4]}_{metadatum[5]}"
    norm = metadatum[3]
    depth = metadatum[1]

    d[alg][ham][evt][norm].append(int(depth))
pprint_nested_dict(format_ddict(d, sort_lists=True))

In [ ]:
# Getting data for hemisphere method
hemisphere_bitstrings = np.empty_like(split_inds, dtype="U6")
for ind, invm_evts in enumerate(split_evts):
    bitstrings = run_hemisphere(invm_evts.reshape(len(invm_evts), -1))
    hemisphere_bitstrings[ind] = [
        "".join(bs) for bs in bitstrings.astype(int).astype(str)
    ]

hemisphere_eff = (
    np.sum(
        (hemisphere_bitstrings == CORRECT) | (hemisphere_bitstrings == swap(CORRECT)),
        axis=1,
    )
    / 2000
)

In [ ]:
print("Invariant mass bins:", ", ".join([str(invm) for invm in data[0].keys()]))
print(
    "Keys:\n ",
    "\n  ".join(
        [
            x
            for x in data[0][1.00].keys()
            if x not in ["alphas", "betas", "gammas", "thetas"]
        ]
    ),
)

---

# VarQITE Normalization Convergence Rates

In [ ]:
varqite_data = [
    datum
    for datum, metadatum in zip(data, metadata)
    if metadatum[0] == "varqite" and metadatum[2] == "H2"
]
varqite_metadata = [
    metadatum
    for metadatum in metadata
    if metadatum[0] == "varqite" and metadatum[2] == "H2"
]
norms = np.array(varqite_metadata)[:, 3]
print(" " * 7, " ".join(norms))
for invm in INVMS[:-1]:
    print(f"{invm:.2f} -- ", end="")
    for datum_param, datum in zip(varqite_metadata, varqite_data):
        print(f"{datum[invm]['evals'].mean():.0f}", end=" ")
    print()
print("-" * 28, "\ntotal - ", end="")
for datum in varqite_data:
    tot_means = np.mean([datum[invm]["evals"].mean() for invm in INVMS[:-1]])
    print(f"{tot_means:.0f}", end=" ")


fig = plt.figure(figsize=(15, 10))
bax = brokenaxes(
    ylims=((0, 500), (7750, 8000), (10250, 10500)),
    hspace=0.1,
)
zorders = [10, 1, 9, 3, 4]
for datum, norm, zorder in zip(varqite_data, norms, zorders):
    evals = np.array([datum[invm]["evals"] for invm in INVMS[:-1]]).flatten()
    bax.hist(
        evals,
        bins=100,
        histtype="stepfilled",
        label=norm.capitalize(),
        alpha=0.7,
        zorder=zorder,
    )

bax.set_xlabel("Steps used", labelpad=30)
bax.set_ylabel("Number of Events", labelpad=70)
bax.set_xlim(0, 500)
bax.legend(loc="upper left")
bax.grid(axis="y");

---

# Efficiency Plots

In [ ]:
def init_efficiency_plot(ax):
    ax.set_xlim(INVMS[0], INVMS[-1])
    ax.set_ylim(0, 1)

    minor_delta = np.diff(INVMS).min()
    ax.set_xticks(INVMS)
    ax.xaxis.set_minor_locator(MultipleLocator(minor_delta))
    ax.yaxis.set_major_locator(MultipleLocator(0.25))
    ax.yaxis.set_minor_locator(MultipleLocator(0.05))
    ax.set_ylabel("Efficiency")
    ax.set_xlabel("$m_{tt}/2m_t$")
    ax.grid(which="major", ls=":")
    ax.set_box_aspect(1)


def make_efficiency_plot(ax, efficiency, fill=False, fill_kwargs={}, **kwargs):
    X = np.repeat(INVMS, 2)[1:-1]
    Y = np.repeat(efficiency, 2)

    ax.plot(X, Y, **kwargs)
    if fill:
        ax.fill_between(X, Y, np.ones_like(Y), **fill_kwargs)

---

## Custom Stuff

In [ ]:
# Find the indices of the other wanted runs
etype = "ttbar"
dtype = "parton"
infos = [
    ["qaoa", "8", "max", etype, dtype],
    ["xqaoa", "8", "max", etype, dtype],
    ["maqaoa", "8", "max", etype, dtype],
    ["falqon", "250", "max", etype, dtype],
    ["varqite", "max", etype, dtype],
]
inds = parse_with_metadata(infos=infos, metadata=metadata, one_per=False)
used_data = data[inds]
used_metadata = metadata[inds]
# Get efficiencies
effs = find_efficiency(data=used_data)
# Adding QA efficiencies by hand
annealer_eff = [0.014, 0.321, 0.812, 0.930, 0.970, 0.979]

In [ ]:
ham = "H2"
legend = False

fig2, ax2 = plt.subplots(figsize=(9, 9))
fig1, ax1 = plt.subplots(figsize=(9, 9))

init_efficiency_plot(ax=ax1)
init_efficiency_plot(ax=ax2)
eff_conf_str, label_buffer = "", 15
for eff, datum, metadatum in zip(effs, used_data, used_metadata):
    if not (metadatum[2] == ham):
        continue

    color = COLORS[metadatum[0]]
    ls = LSS[metadatum[0]]
    depth = f"($p={metadatum[1]}$)" if metadatum[0] != "varqite" else ""
    label = f"{LABELS[metadatum[0]]} {depth}\n$\\epsilon={100 * np.mean(eff):.2f}$%"
    make_efficiency_plot(
        ax=ax1, efficiency=eff, c=color, lw=4, ls=ls, label=label, zorder=1
    )

    eff_conf_str += f"{label.split('-')[0].strip().replace('$', ''):<{label_buffer}} - "
    eff_conf_str += (
        " ".join([f"{2 * datum[invm]['rank_probs'].mean():.2f}" for invm in INVMS[:-1]])
        + "\n"
    )

    rank_probs = np.array([datum[invm]["rank_probs"].mean() for invm in INVMS[:-1]])
    label = f"{metadatum[0]} {depth} -- {100 * 2 * np.mean(rank_probs):.2f}%"
    make_efficiency_plot(ax=ax2, efficiency=2 * rank_probs, lw=4, label=label)

# Plot efficiency for hemisphere method
make_efficiency_plot(
    ax=ax1,
    efficiency=hemisphere_eff,
    c=COLORS["hemisphere"],
    ls=LSS["hemisphere"],
    lw=4,
    label=f"Hemisphere Method\n$\\epsilon={100 * np.mean(hemisphere_eff):.2f}$%",
    zorder=0,
)
# Plot quantum annealer data
make_efficiency_plot(
    ax=ax1,
    efficiency=annealer_eff,
    c=COLORS["annealer"],
    ls=LSS["annealer"],
    lw=4,
    label=f"Quantum Annealer\n$\\epsilon={100 * np.mean(annealer_eff):.2f}$%",
    zorder=0,
)

buffer = max([len(x.split("- ")[0]) for x in eff_conf_str.split("\n")]) + 2
print(" " * buffer + " ".join([f"{invm:.2f}" for invm in INVMS[:-1]]))
print(eff_conf_str)

make_efficiency_plot(
    ax=ax1,
    efficiency=bruteforce_dict[ham],
    color=FILL_KWARGS["color"],
    zorder=0,
    fill=True,
    fill_kwargs=FILL_KWARGS,
)
ax1.set_title(rf"Hamiltonian ${'_'.join(list(ham))}$")
ax2.set_ylabel("Average probability")
ax2.set_title(rf"Mean Probabilities for Hamiltonian: ${'_'.join(list(ham))}$")
# ax1.text(2.51, 0.02, "Smeared Events", fontsize=14)

if legend:
    ax1.legend(loc="lower right")
    ax2.legend(loc="lower right")

In [ ]:
handles = []
for metadatum, eff in zip(used_metadata, effs):
    if metadatum[2] != ham:
        continue

    depth = f"($p={metadatum[1]}$)" if metadatum[0] != "varqite" else ""
    label = f"{LABELS[metadatum[0]]} {depth}\n$\\epsilon={100 * np.mean(eff):.2f}$%"
    color = COLORS[metadatum[0]]
    ls = LSS[metadatum[0]]
    lw = 4
    handles.append(Line2D([0], [0], label=label, color=color, ls=ls, lw=lw))

handles.append(
    Line2D(
        [0],
        [0],
        label=f"{LABELS['hemisphere']}\n$\\epsilon={100 * np.mean(hemisphere_eff):.2f}$%",
        color=COLORS["hemisphere"],
        ls=LSS["hemisphere"],
        lw=lw,
    )
)
# handles.append(
#     Line2D(
#         [0],
#         [0],
#         label=f"{LABELS['annealer']}\n$\\epsilon={100 * np.mean(annealer_eff):.2f}$%",
#         color=COLORS["annealer"],
#         ls=LSS["annealer"],
#         lw=lw,
#     )
# )

fig = plt.figure()
leg = fig.legend(handles=handles, loc="center", handlelength=4, ncols=7)
fig.patch.set_visible(False)
plt.axis("off")
plt.tight_layout()

---

## FALQON Depths

In [ ]:
ham = "H2"
depths = [25, 50, 100, 250, 500, 1000, 1500, 2000, 2500]
infos = [["falqon", "ttbar", "parton", ham, str(depth)] for depth in depths]
falqon_inds = parse_with_metadata(infos, metadata=metadata)
falqon_data = data[falqon_inds]
falqon_metadata = metadata[falqon_inds]
falqon_effs = find_efficiency(data=falqon_data)

In [ ]:
fig2, ax2 = plt.subplots(figsize=(9, 9))
fig1, ax1 = plt.subplots(figsize=(9, 9))

init_efficiency_plot(ax=ax1)
init_efficiency_plot(ax=ax2)
eff_conf_str, label_buffer = "", 15
for eff, datum, metadatum in zip(falqon_effs, falqon_data, falqon_metadata):
    ls = "solid"
    label = f"[$p={metadatum[1]}$] -- {100 * np.mean(eff):.2f}%"
    make_efficiency_plot(ax=ax1, efficiency=eff, ls=ls, lw=4, label=label)

    eff_conf_str += f"{label.split('-')[0].strip().replace('$', ''):<{label_buffer}} - "
    eff_conf_str += (
        " ".join([f"{2 * datum[invm]['rank_probs'].mean():.2f}" for invm in INVMS[:-1]])
        + "\n"
    )
    rank_probs = np.array([datum[invm]["rank_probs"].mean() for invm in INVMS[:-1]])
    label = f"[$p={metadatum[1]}$] -- {100 * 2 * np.mean(rank_probs):.2f}%"
    make_efficiency_plot(ax=ax2, efficiency=2 * rank_probs, ls=ls, lw=4, label=label)

buffer = max([len(x.split("- ")[0]) for x in eff_conf_str.split("\n")]) + 2
print(" " * buffer + " ".join([f"{invm:.2f}" for invm in INVMS[:-1]]))
print(eff_conf_str)

fill_kwargs = {"color": "#23578923", "hatch": "x"}
make_efficiency_plot(
    ax=ax1,
    efficiency=bruteforce_dict[ham],
    color="#235789",
    fill=True,
    fill_kwargs=fill_kwargs,
)
ax1.set_title(rf"Efficiency for ${'_'.join(list(ham))}$")
ax1.legend(loc="lower right")
ax2.set_ylabel("Average probability")
ax2.set_title(r"Mean Probabilities for $H_2$")
ax2.legend()

---

# Mass Histograms

## 2-Dimensional

In [ ]:
def plot_2dhist_mass(masses, etype, bins=250, lims=(0, 500), m_6jet=0):
    # Mask the masses to get constant sized bins on plot
    lim_mask = ((masses > lims[0]) & (masses < lims[1])).all(axis=1)
    fig, ax = plt.subplots(figsize=(12, 12))
    divider = make_axes_locatable(ax)
    ax_cbar = divider.append_axes("right", size="5%", pad=0.15)
    fig.add_axes(ax_cbar)

    _, _, _, img = ax.hist2d(
        *masses[lim_mask].T,
        bins=bins,
        norm=LogNorm(vmin=1, vmax=1e3),
        cmap="RdPu",
        range=(lims, lims),
    )
    cbar = fig.colorbar(img, cax=ax_cbar)
    cbar.set_label("Counts", rotation=270, labelpad=15)

    yaxis_mass = {"tW": W_MASS, "ttbar": TOP_MASS, "6jet": m_6jet}[etype]
    yaxis_label = {"tW": "m_W", "ttbar": "m_t", "6jet": "m"}[etype]
    xaxis_mass = {"tW": TOP_MASS, "ttbar": TOP_MASS, "6jet": m_6jet}[etype]
    xaxis_label = {"tW": "m_t", "ttbar": "m_t", "6jet": "m"}[etype]
    origin = {
        "tW": [TOP_MASS, W_MASS],
        "ttbar": [TOP_MASS, TOP_MASS],
        "6jet": [m_6jet, m_6jet],
    }[etype]
    ax.axvline(xaxis_mass, c="k", ls=":", alpha=0.4, zorder=10)
    ax.axhline(yaxis_mass, c="k", ls=":", alpha=0.4, zorder=10)

    ax.set_ylabel(f"${yaxis_label}$ (GeV)")
    ax.yaxis.set_major_locator(MultipleLocator(50))
    ax.set_xlabel(f"${xaxis_label}$ (GeV)")
    ax.xaxis.set_major_locator(MultipleLocator(50))
    ax.grid(alpha=0.3)

    fracs = [0.8, 0.9, 0.99]
    lss = [(0, (1, 1)), (0, (5, 5)), "solid"]
    alpha = "aa"
    colors = [f"#8642A8{alpha}", f"#8642A8{alpha}", f"#8642A8{alpha}"]
    for frac, ls, color in zip(fracs, lss, colors):
        distances = np.sqrt(np.sum((masses - origin) ** 2, axis=1))
        sorted_distances = np.sort(distances)
        thresh = sorted_distances[int(frac * len(sorted_distances))]
        circle = plt.Circle(origin, thresh, ec=color, ls=ls, fc="#00000000", lw=2)
        ax.add_patch(circle)

    handles = [
        Line2D([0], [0], ls=ls, label=f"{100 * frac:.0f}%", c=c)
        for ls, c, frac in zip(lss, colors, fracs)
    ]
    ax.legend(handles=handles)

In [ ]:
info = ["xqaoa", "8", "max", "H2", "ttbar", "smeared"]
ind = parse_with_metadata(infos=[info], metadata=metadata)[0]
masses = get_2dhist_invms(data[ind], metadata[ind])
print(metadata[ind])
if metadata[ind][4] != "6jet":
    print(f"Efficiency: {100 * np.mean(find_efficiency(data=[data[ind]])):.2f}%")
plot_2dhist_mass(masses, bins=300, lims=(0, 500), etype=metadata[ind][4])

In [ ]:
# `evts` is defined by etype and dtype at the very beginning
masses = get_bitstring_invms(evts, bitstrings=CORRECT)
print("correct bitstring")
plot_2dhist_mass(masses, lims=(0, 500), etype=evts_etype)

In [ ]:
ham = "H2"
min_bitstrings = get_minimum_energies(evts=evts, hamiltonian=ham)[:, 0].astype("S6")
masses = get_bitstring_invms(evts=evts, bitstrings=min_bitstrings)
print(f"{ham} Hamiltonian")
print(
    f"Efficiency: {100 * np.sum(min_bitstrings.astype('U6') == CORRECT) / len(min_bitstrings):.2f}%"
)
plot_2dhist_mass(masses, lims=(0, 500), etype=metadata[ind][4])

In [ ]:
masses = get_bitstring_invms(
    evts=evts, bitstrings=hemisphere_bitstrings.astype("S6").flatten()
)
print("Hemisphere method")
print(
    f"Efficiency: {np.sum(np.logical_or(hemisphere_bitstrings == CORRECT, hemisphere_bitstrings == swap(CORRECT))) / len(hemisphere_bitstrings.flatten()):.2f}"
)
plot_2dhist_mass(masses, lims=(0, 500), etype=metadata[ind][4])

## 1-Dimensional

In [ ]:
ham = "H0"
algs = ["qaoa", "maqaoa", "xqaoa", "varqite"]
infos = [
    ["maqaoa", "max", ham, "8", "parton"],
    ["varqite", "max", ham, "parton"],
    ["falqon", "max", ham, "250", "parton"],
    ["xqaoa", "max", ham, "8", "parton"],
    ["qaoa", "max", ham, "8", "parton"],
]
inds = parse_with_metadata(infos=infos, metadata=metadata)
all_masses = np.array([get_2dhist_invms(data[ind], metadata[ind]) for ind in inds])

fig, ax = plt.subplots(figsize=(15, 8))
# This will make sure they all have the same mass bin width
bins = np.histogram(np.hstack(all_masses), bins=1000)[1]
# This centers the bins on the TOP_MASS, looks nice :)
bins -= (bins[np.searchsorted(bins, TOP_MASS)] - TOP_MASS) + (bins[1] - bins[0]) / 2
handles = []
for masses, info in zip(all_masses, infos):
    ax.hist(masses.flatten(), histtype="step", bins=bins, color=COLORS[info[0]])
    depth_label = "" if not info[3].isdigit() else f" ($p={info[3]}$)"
    label = f"{LABELS[info[0]]}{depth_label}"
    handles.append(Patch(label=label, color=COLORS[info[0]]))
ax.axvline(TOP_MASS, c="k", ls=":", alpha=0.4, zorder=10)

diff = 10
ax.set_xlim(TOP_MASS - diff, TOP_MASS + diff)
ax.set_xlabel("$m_t$ (GeV)")
# ax.set_yscale("log")
ax.set_ylabel("Counts")
ax.legend(handles=handles)

---

# Number of Jets Histogram

In [ ]:
def plot_hist_numjets(jet_vals, jet_counts, num_fsp):
    fig, ax = plt.subplots(figsize=(12, 12))

    bar = ax.bar(jet_vals, jet_counts / jet_counts.sum(), width=0.8, color="#7b3e19")

    ax.xaxis.set_major_locator(MultipleLocator(1))
    ax.set_xlim(-0.5, num_fsp + 0.5)
    ax.set_xlabel("Number of Jets")

    ax.yaxis.set_major_locator(MultipleLocator(0.1))
    ax.set_ylim(0, 1)
    ax.set_ylabel("Fraction of Events")
    ax.tick_params(axis="y", right=True)
    ax.grid(axis="y", alpha=0.4)

    ax.bar_label(bar, fmt="{:.3f}")
    ax.set_axisbelow(True)
    ax.set_box_aspect(1)

In [ ]:
info = ["falqon", "250", "H2", "max", "ttbar", "parton"]
ind = parse_with_metadata([info], metadata=metadata)[0]
datum = data[ind]
bitstrings = get_bitstrings(datum)
num_fsp = len(bitstrings[0][0])
num_jets0 = np.char.count(bitstrings, "0").flatten()
num_jets1 = np.char.count(bitstrings, "1").flatten()
jet_vals, jet_counts = np.unique(
    np.concatenate((num_jets0, num_jets1)), return_counts=True
)

print(metadata[ind])
plot_hist_numjets(jet_vals, jet_counts, num_fsp=num_fsp)

In [ ]:
ham = "H2"
bitstrings = get_minimum_energies(evts=evts, hamiltonian=ham)[:, 0].astype(str)
num_fsp = len(bitstrings[0])
num_jets0 = np.char.count(bitstrings, "0").flatten()
num_jets1 = np.char.count(bitstrings, "1").flatten()
jet_vals, jet_counts = np.unique(
    np.concatenate((num_jets0, num_jets1)), return_counts=True
)

print(f"Hamiltonian: {ham}")
plot_hist_numjets(jet_vals, jet_counts, num_fsp=num_fsp)

In [ ]:
num_fsp = len(hemisphere_bitstrings[0][0])
num_jets0 = np.char.count(hemisphere_bitstrings, "0").flatten()
num_jets1 = np.char.count(hemisphere_bitstrings, "1").flatten()
jet_vals, jet_counts = np.unique(
    np.concatenate((num_jets0, num_jets1)), return_counts=True
)

plot_hist_numjets(jet_vals, jet_counts, num_fsp=num_fsp)

---

# Efficiency vs Depth

In [ ]:
ham = "H2"
depths = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 25]
algs = ["qaoa", "maqaoa", "xqaoa"]
# Get all depths for the given algorithsm
infos = np.array(
    [[[alg, depth, "max", ham, "ttbar", "parton"] for depth in depths] for alg in algs]
).reshape(len(algs) * len(depths), -1)
# Get their indices and find the efficiencies
inds = parse_with_metadata(infos=infos, metadata=metadata)
correct_effs = np.mean(find_efficiency(data=data[inds]), axis=1).reshape(len(algs), -1)
min_effs = np.mean(find_efficiency(data=data[inds], find_correct=False), axis=1).reshape(
    len(algs), -1
)
# Get best case for correct efficiency
min_bitstrings = get_minimum_energies(evts=evts, hamiltonian=ham)[:, 0]
ham_limit_eff = np.sum(min_bitstrings == CORRECT) / len(min_bitstrings)
# Get VarQITE values
varqite_datum = data[
    parse_with_metadata(infos=[["varqite", "max", "H2", "parton"]], metadata=metadata)
]
varqite_correct = np.mean(find_efficiency(data=varqite_datum))
varqite_min = np.mean(find_efficiency(data=varqite_datum, find_correct=False))

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))

# Plots algs
for alg, correct_eff, min_eff in zip(algs, correct_effs, min_effs):
    color = COLORS[alg]
    ax.plot(depths, correct_eff, marker="o", ls="dotted", c=color)
    ax.plot(depths, min_eff, marker="o", ls="solid", c=color)

# Plot VarQITE
ax.axhline(varqite_correct, ls="dotted", c=COLORS["varqite"], lw=3)
ax.axhline(varqite_min, ls="solid", c=COLORS["varqite"], lw=3)

# Plot Hamiltonian limit
xlim = [depths[0] - 0.5, depths[-1] + 0.5]
ax.fill_between(xlim, ham_limit_eff, 1, **FILL_KWARGS)
ax.axhline(ham_limit_eff, c=FILL_KWARGS["color"], alpha=0.8, lw=2, zorder=0)

ax.yaxis.set_major_locator(MultipleLocator(0.2))
ax.yaxis.set_minor_locator(MultipleLocator(0.1))
ax.set_xticks(depths)
ax.set_xlim(*xlim)
ax.set_xlabel("Depth $p$")
ax.set_ylim(0, 1)
ax.set_ylabel("Success Rate")
ax.grid(axis="y")
ax.grid(axis="y", which="minor", ls="dotted")

eff_handles = [
    Line2D([0], [0], ls="solid", label="Minimum", color="k"),
    Line2D([0], [0], ls="dotted", label="Correct", color="k"),
]
eff_leg = ax.legend(
    handles=eff_handles,
    bbox_to_anchor=(0.840, 0),
    loc="lower right",
    title="Efficiency Type",
)
ax.add_artist(eff_leg)

alg_handles = [Patch(color=COLORS[alg], label=LABELS[alg]) for alg in algs + ["varqite"]]
ax.legend(handles=alg_handles, title="Algorithm", loc="lower right")

## FALQON

In [ ]:
# Get full info of FALQON run
ham = "H2"
etype = "ttbar"
num_evts = 2000
full_falqon_data = load_data(
    alg="falqon",
    depth=2500,
    hamiltonian=ham,
    norm="max",
    etype=etype,
    dtype="parton",
    lambda_nume="minJij",
    lambda_denom="maxPij",
    num_evts=num_evts,
)
# Additional info
cor_bs = SYM_TRUE_BS_DICT[etype]
num_fsp = NUM_FSP_DICT[etype]
bitstrings = np.array([format(x, f"0{num_fsp}b") for x in range(2**num_fsp)])
min_bitstrings = get_minimum_energies(evts=evts, hamiltonian=ham)[:, 0]
ham_limit_eff = np.sum(min_bitstrings == CORRECT) / len(min_bitstrings)

eff_cor_depth = np.zeros(2500)
eff_min_depth = np.zeros(2500)
# Iterate over invariant masses and get efficiencies
for invm in INVMS[:-1]:
    # Find order of probabilities by index
    prob_ordered_inds = np.flip(
        np.argsort(full_falqon_data[invm]["depth_probs"], axis=2), axis=2
    )
    # Get minimum bitstring for each event, cast to correct array shape
    min_bs = get_minimum_energies(
        evts=full_falqon_data[invm]["invm_p4s"], hamiltonian=ham
    )[:, 0][:, None, None]
    # Find where bitstring of importance is
    ranks_cor = np.argmax(bitstrings[prob_ordered_inds] == cor_bs, axis=-1)
    ranks_min = np.argmax(bitstrings[prob_ordered_inds] == min_bs, axis=-1)
    # Assume symmetry
    ranks_cor -= ranks_cor % 2
    ranks_min -= ranks_min % 2
    # And find if it ranks first
    eff_cor_depth += np.count_nonzero(ranks_cor == 0, axis=0)
    eff_min_depth += np.count_nonzero(ranks_min == 0, axis=0)
# Free up RAM
del full_falqon_data
# Normalize
tot_num_evts = int(num_evts) * (len(INVMS) - 1)
eff_cor_depth /= tot_num_evts
eff_min_depth /= tot_num_evts

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

xlim = [10, 2500]
ax.fill_between(xlim, ham_limit_eff, 1, **FILL_KWARGS)
ax.axhline(ham_limit_eff, c=FILL_KWARGS["color"], alpha=0.8, lw=2, zorder=0)

ax.plot(eff_cor_depth, c="#48156F", lw=4, label="Correct")
ax.plot(eff_min_depth, c="#2DABAF", ls="dashed", lw=4, label="Minimum")
ax.plot(ham_limit_eff * eff_min_depth, c="k", lw=4, alpha=0.2, label="Perfect")
ax.set_ylim(0, 1)
ax.set_ylabel("Efficiency")
ax.set_xlim(*xlim)
ax.set_xlabel("Depth $p$")
ax.set_xscale("log")
ax.grid(axis="x", which="major")
ax.grid(axis="x", which="minor", ls="dotted")
ax.grid(axis="y", which="major")
ax.legend(loc="lower right")

---

# Exploring $\lambda_2$
If we write $H_2=H_0 + \dfrac{\lambda}{2\lambda_2}H_1$, then $\lambda_2$ acts as a scaling factor that shifts between $H_0$ and $H_1$, with $\lambda_2=1$ giving $H_2$.

In [ ]:
num_lmbdas = 120
min_lambda2, max_lambda2 = 10**-4, 10**6
lambda2s = np.geomspace(min_lambda2, max_lambda2, num_lmbdas)

# Find efficiency of Hamiltonian for different values of lambda2
effs = []
for invm_evts in split_evts:
    invm_effs = []
    for lambda2 in lambda2s:
        min_bitstrings = get_minimum_energies(
            evts=invm_evts, hamiltonian="H2", scale=1 / lambda2
        )[:, 0]
        invm_effs.append(np.sum(min_bitstrings == CORRECT) / len(min_bitstrings))
    effs.append(invm_effs)
effs = np.array(effs)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))

lambda_label = r"\lambda=\min(J_{ij})/\max(P_{ij})"
colors = multifader(
    colors=["#5D2E46", "#E53D00"], fractions=np.linspace(0, 1, len(INVMS) - 1)
)
for invm_effs, color, invm_lo, invm_hi in zip(effs, colors, INVMS[:-1], INVMS[1:]):
    label = f"${invm_lo:.2f}<m_{{tt}}/2m_t<{invm_hi:.2f}$"
    # ax.scatter(lambda2s, invm_effs, c=color)
    ax.plot(lambda2s, invm_effs, c=color, label=label)

ax.plot(
    lambda2s,
    effs.mean(axis=0),
    c="k",
    alpha=0.7,
    ls=(5, (10, 3)),
    lw=4,
    label="Total",
    dash_capstyle="round",
)
max_ind = effs.mean(axis=0).argmax()
max_eff_lambda2 = lambda2s[max_ind]
max_eff = effs.mean(axis=0)[max_ind]
ax.axvline(max_eff_lambda2, c="k", ls="dotted", alpha=0.5)
ax.axhline(max_eff, c="k", ls="dotted", alpha=0.5)
val, power = f"{max_eff_lambda2:.3e}".split("e+")
v = rf"{val}x10^{{{int(power)}}}"
ax.annotate(
    "Optimal\n" + rf"$\lambda_2={v}$" + "\n" + rf"$\epsilon={max_eff:.2f}$",
    xy=(max_eff_lambda2, max_eff),
    xytext=(max_eff_lambda2 / 8.5, max_eff / 1.3),
    ha="center",
    arrowprops={"fc": "black", "shrink": 0.05},
    bbox={"boxstyle": "round", "fc": "white"},
)

ax.axvline(1, c="k", lw=4, zorder=0)
ax.text(
    s="$H_2$",
    x=1.3,
    y=0.55,
    fontsize=15,
    bbox={"boxstyle": "round", "fc": "#dddddd"},
)
right = dict(boxstyle="rarrow,pad=0.3", fc="#dddddd", lw=2)
left = dict(boxstyle="larrow,pad=0.3", fc="#dddddd", lw=2)
ax.text(
    0.88,
    0.55,
    f"{' ' * 6}$H_0${' ' * 6}",
    size=15,
    bbox=right,
    transform=ax.transAxes,
)
ax.text(
    0.04,
    0.55,
    f"{' ' * 6}$H_1${' ' * 6}",
    size=15,
    bbox=left,
    transform=ax.transAxes,
)

ax.set_ylabel("Efficiency", fontsize=15)
ax.set_ylim(0, 1.02)
ax.yaxis.set_major_locator(MultipleLocator(0.1))
ax.yaxis.set_minor_locator(MultipleLocator(0.05))
ax.set_xscale("log")
ax.xaxis.set_major_locator(LogLocator(numticks=10))
ax.xaxis.set_minor_locator(LogLocator(numticks=1000, subs="auto"))
ax.set_xlim(0.95 * min_lambda2, 1.05 * max_lambda2)
ax.set_xlabel(r"$\lambda_2$", fontsize=15)
ax.set_title(f"{evts_dtype.capitalize()} events, ${lambda_label}$", fontsize=18)
ax.grid(alpha=0.6)
ax.grid(which="minor", alpha=0.6, ls="dotted")
ax.legend()

In [ ]:
comp_evts = get_data("ttbar", "parton")[0]
split_comp_evts = split_data(comp_evts)[0]
eff_types = [
    ["H2", "$H_2$", {}],
    ["H2", "$H_2$ optimal", {"scale": 1 / max_eff_lambda2}],
    ["H2", "$H_2$ not so optimal", {"scale": 1 / 100}],
    ["H0", "$H_0$", {}],
]

fig, ax = plt.subplots(figsize=(8, 8))
init_efficiency_plot(ax)
for ham, label, kwargs in eff_types:
    effs = []
    for cevts in split_comp_evts:
        effs.append(
            (
                get_minimum_energies(cevts, hamiltonian=ham, **kwargs)[:, 0] == CORRECT
            ).sum()
            / len(cevts)
        )

    make_efficiency_plot(
        ax=ax, efficiency=effs, lw=4, label=f"{label} - {100 * np.mean(effs):.2f}%"
    )
ax.legend()


---

# Per Event VarQITE Analysis

In [ ]:
coeffs = get_coefficients(
    hamiltonian="H2", evts=evts, nume=["min", "Jij"], denom=["max", "Pij"]
)[split_inds]
min_energies = get_minimum_energies(
    hamiltonian="H2", evts=evts, nume=["min", "Jij"], denom=["max", "Pij"]
)[split_inds]
all_energies = get_all_bitstring_energies(
    evts=evts, hamiltonian="H2", nume=["min", "Jij"], denom=["max", "Pij"]
)[1][split_inds]

# 0 -- 1.00 | 1 -- 1.25 | 2 -- 1.50
# 3 -- 1.75 | 4 -- 2.00 | 5 -- 2.50
invm_bin = 2
N = 0

coeff = coeffs[invm_bin][N] / coeffs[invm_bin][N].max()
min_energy = min_energies[invm_bin][N]
all_energy = all_energies[invm_bin][N]
bitstring_energies = all_energies[invm_bin][N]
invm = invms[split_inds][invm_bin][N]

In [ ]:
print(f"Minimum bitstring: {min_energy[0]} -- {min_energy[1]:+.3e}")

varqite = VarQITE(coeff=coeff, dtau=0.5, steps=500, prec=1e-5, device="lightning.gpu")
varqite.optimize(steps_till_newline=10)

In [ ]:
correct_bs = "000111"
fig, ax = plt.subplots(figsize=(18, 8))

probs = varqite.get_probs()
probs_plot = probs
bar = bar_count(
    ax=ax,
    counts=probs_plot,
    sort_type="desc",
    bar_params={"color": "#333355"},
    x_rot=90,
)
ax.set_ylabel("Probability")
red_inds = np.unique(
    np.append(
        np.where(bar.datavalues == probs_plot[correct_bs]),
        np.where(bar.datavalues == probs_plot.get(swap(correct_bs))),
    )
)
green_inds = np.unique(
    np.append(
        np.where(bar.datavalues == probs_plot[min_energy[0]]),
        np.where(bar.datavalues == probs_plot.get(swap(min_energy[0]))),
    )
)
for red_ind, green_ind in zip(red_inds, green_inds):
    if red_ind == green_ind:
        bar[red_ind].set_color("#D0DE9C")
    else:
        bar[red_ind].set_color("#CB1616")
        bar[green_ind].set_color("#16DB93")

ax.annotate(
    text=f"Minimum bitstring: {min_energy[0]}\nInvariant mass: {invm:.2f}",
    xy=(0.8, 0.9),
    xycoords="axes fraction",
    bbox=dict(boxstyle="round", facecolor="#99a7cb"),
)
# ax.set_yscale("log")

eng_ax = ax.twinx()
# Plot the energy for each bitstring
all_energy_dict = dict(zip(varqite.bitstrings.tolist(), all_energy.tolist()))
prob_energies = (
    np.array([all_energy_dict[t.get_text()] for t in ax.get_xticklabels()]) / 1e9
)
num_x = len(prob_energies)
xrange = np.arange(num_x)


# fit energies to a quadratic
def fit(x, a, b, c):
    return a * x**2 + b * x + c


params = curve_fit(f=fit, xdata=xrange, ydata=prob_energies)[0]
eng_diff = abs(prob_energies.max() - prob_energies.min())
x_fit = np.linspace(-0.6, num_x, 100)
y_fit = fit(x_fit, *params)

eng_ax.plot(prob_energies, c="k", ls="dashed", alpha=0.3)
eng_ax.plot(
    xrange[prob_energies > 0],
    prob_energies[prob_energies > 0],
    marker="d",
    lw=0,
    markerfacecolor="none",
    markeredgecolor="k",
    markeredgewidth=1.5,
)
eng_ax.plot(
    xrange[prob_energies < 0],
    prob_energies[prob_energies < 0],
    marker="d",
    lw=0,
    color="k",
)
eng_ax.plot(x_fit, y_fit, c="#83a18f", alpha=0.7, lw=5, zorder=0)
eng_ax.set_ylabel("Energy [GeV]")
eng_ax.axhline(0, ls="dotted", alpha=0.1, c="k")
eng_ax.set_ylim(
    prob_energies.min() - 0.10 * eng_diff, prob_energies.max() + 0.10 * eng_diff
)
eng_ax.fill_between(x=x_fit, y1=0, y2=eng_ax.get_ylim()[0], color="k", alpha=0.1)
ax.set_xlim(-0.6, num_x - 0.4);

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))

all_probs = np.array(
    [list(varqite.get_probs(thetas).values()) for thetas in varqite.all_thetas]
)
for ind, probs in enumerate(all_probs.T):
    bitstring = format(ind, f"0{varqite.nq}b")
    ls = "solid" if bitstring in [correct_bs, swap(correct_bs)] else "dotted"
    ax.plot(probs, ls=ls)
ax.set_title("Probabilities per step")
ax.set_xlim(0, varqite.total_steps - 1)
ax.set_xlabel("steps")
ax.set_ylim(0, 0.5)
ax.set_ylabel("probability")
# ax.set_yscale("log")

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))

ax.plot(varqite.energy_diffs, c="k", label=r"$\Delta E$")
ax.plot(
    np.abs(varqite.energy_diffs / varqite.energies[1:]),
    c="#b9a0b1",
    label=r"$\Delta E / E$",
)
ax.axhline(varqite.prec, c="#b9a0b1", ls="dashed", label="Precision limit")
ax.set_xlim(0, varqite.total_steps - 2)
ax.set_yscale("log")
ax.set_xlabel("steps")
ax.set_title(r"$\Delta E$ per step")
ax.legend(loc="upper right")

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))

min_energy = min(varqite.energies.min(), varqite.op_energies.min())

for op_energies in varqite.op_energies.T:
    ax.plot(op_energies - min_energy, ls="dotted")
ax.plot(varqite.energies - min_energy, c="k", lw=2)
ax.set_yscale("log")
ax.set_ylabel("Normalized energy")
ax.axhline(-min_energy, c="k", alpha=0.2)
ax.set_xlim(0, varqite.total_steps - 1)
ax.set_xlabel("steps")
ax.set_title("Energies per step")

# ax2 = ax.twinx()
# ax2.plot(varqite.energies - varqite.energies.min(), c="r", ls="dashed")
# ax2.set_yscale("log")
handles = [
    # Line2D([0], [0], c="none", label="Left Axis"),
    Line2D([0], [0], ls="dotted", c="k", label="Pauli-$Z$ strings"),
    Line2D([0], [0], ls="solid", c="k", label="Hamiltonian"),
    Line2D([0], [0], ls="solid", c="k", alpha=0.2, label="Shifted zero energy"),
    # Line2D([0], [0], c="none", label="Right Axis"),
    # Line2D([0], [0], ls="dashed", c="r", label="Hamiltonian"),
]
ax.legend(handles=handles, loc="lower left")  # , ncol=2)

In [ ]:
def pi_formatter(x, pos):
    frac = Fraction(x / np.pi).limit_denominator()
    num, den = frac.numerator, frac.denominator

    if num == 0:
        return "0"
    elif num == den:
        return r"$\pi$"
    elif den == 1:
        return rf"${num}\pi$"
    elif num == 1:
        return rf"$\frac{{\pi}}{{{den}}}$"
    else:
        return rf"$\frac{{{num}\pi}}{{{den}}}$"


fig, ax = plt.subplots(figsize=(15, 8))

for thetas in np.array(varqite.all_thetas).T:
    ax.plot(thetas)
ax.set_xlim(0, varqite.total_steps - 1)
# ax.set_ylim(0, 2 * np.pi)
# ax.set_ylim(np.pi - np.pi / 2, np.pi + np.pi / 2)
ax.yaxis.set_major_locator(MultipleLocator(np.pi / 4))
ax.yaxis.set_minor_locator(MultipleLocator(np.pi / 8))
ax.yaxis.set_major_formatter(FuncFormatter(pi_formatter))
ax.set_title("Parameters per step")
ax.set_xlabel("steps")
ax.grid(axis="y", which="major")
ax.grid(axis="y", which="minor", ls="dotted")

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))

for Dvec_ele in np.array(varqite.Dvecs).T:
    ax.plot(Dvec_ele)
# ax.set_yscale("log")
# ax.set_ylim(10e-10, 1)
ax.set_xlim(0, varqite.total_steps - 1)
ax.set_xlabel("steps")
ax.set_title("Elements of $D$ per step")

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))

for Gmat_ele in varqite.Gmats.reshape(varqite.total_steps, -1).T:
    ax.plot(Gmat_ele)
ax.set_xlim(0, varqite.total_steps - 1)
ax.set_xlabel("steps")
ax.set_title("Elements of $G$ per step")
ax.grid()

---

# Miscellany

## $\Delta E$ for $H_0$ vs $H_2$

In [ ]:
# Get (12,000, 64) array in bitstring order (e.g. 000000, 000001, etc) of energies
_, engs_H2 = get_all_bitstring_energies(evts=evts, hamiltonian="H2")
_, engs_H0 = get_all_bitstring_energies(evts=evts, hamiltonian="H0")
# Now sorted by energy
sorted_engs_H2 = np.sort(engs_H2, axis=1)
sorted_engs_H0 = np.sort(engs_H0, axis=1)
# Difference between the lowest two energies normalized by the lowest energy
deltaE_H2 = np.abs(
    np.squeeze(np.diff(sorted_engs_H2[:, [0, 2]], axis=1)) / sorted_engs_H2[:, 0]
)
deltaE_H0 = np.abs(
    np.squeeze(np.diff(sorted_engs_H0[:, [0, 2]], axis=1)) / sorted_engs_H0[:, 0]
)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))

thresh = 10
perc_omit = 100 * np.sum(deltaE_H0 > thresh) / len(deltaE_H0)
ax.hist(deltaE_H2[deltaE_H2 < thresh], bins=100, histtype="step", label="$H_2$")
ax.hist(deltaE_H0[deltaE_H0 < thresh], bins=100, histtype="step", label="$H_0$")
ax.set_ylabel("Counts", fontsize=15)
ax.set_xlim(0, thresh)
ax.set_xlabel(r"$\Delta E / E_0$", fontsize=15)
ax.annotate(
    f"{perc_omit:.2f}% of $H_0$ omitted",
    xy=(0.5, 0.9),
    xycoords="axes fraction",
    bbox=dict(fc="white"),
)
ax.legend()

## QNode to QASM

In [ ]:
def export_qasm(qnode, params):
    """
    Convert a (parameterized) QNode into QASM code
    """
    tape = qml.workflow.construct_tape(qnode, level="device")(*params)
    return tape.to_openqasm()

## Decompose Pauli String Gates

In [ ]:
gate_set = {qml.CNOT, qml.RX, qml.RY, qml.RZ, qml.PauliX, qml.PauliY, qml.PauliZ}


def create_circuit(pauli_string, theta_val=0.5):
    """
    Draws the decomposition of the given Pauli string.
    """

    def circuit(theta):
        qml.PauliRot(theta, pauli_string, wires=range(len(pauli_string)))
        return qml.probs()

    qnode = qml.QNode(circuit, device=qml.device("default.qubit"))

    qml.draw_mpl(qml.transforms.decompose(qnode, gate_set=gate_set), decimals=4)(
        theta_val
    )
    return qnode

In [ ]:
qnode = create_circuit("ZZ")

In [ ]:
qnode = create_circuit("ZY")

In [ ]:
qnode = create_circuit("ZYXXZ")